# 人脸检测和识别训练流程

以下示例展示了如何在自己的数据集上微调InceptionResnetV1模型。这将主要遵循标准的PyTorch训练模式。

In [1]:
from pathlib import Path
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

#### 定义运行参数

数据集应该遵循VGGFace2/ImageNet风格的目录布局。将`data_dir`修改为您要微调的数据集所在的位置。

In [2]:
root_dir = Path(Path(__file__).parent if "__file__" in globals() else "").absolute().parent
data_dir = root_dir / "tests" / "data" / "test_images"

batch_size = 32
epochs = 8
workers = 0 if os.name == "nt" else 8

#### 判断是否有nvidia GPU可用

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("在该设备上运行: {}".format(device))

在该设备上运行: cuda:0


#### 定义MTCNN模块

查看`help(MTCNN)`获取更多细节。

In [4]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### 执行MTCNN人脸检测

迭代DataLoader对象并获取裁剪后的人脸。

In [5]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(str(data_dir), str(data_dir.parent / (data_dir.name + "_aligned"))))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print(f"\r第 {i + 1} 批，共 {len(loader)} 批", end="")
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

第 1 批，共 1 批

#### 定义Inception Resnet V1模块

查看`help(InceptionResnetV1)`获取更多细节。

In [6]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained="vggface2",
    num_classes=len(dataset.class_to_idx)
).to(device)

#### 定义优化器、调度器、数据集和数据加载器

In [7]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir.parent / (data_dir.name + "_aligned"), transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### 定义损失和评估函数

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    "fps": training.BatchTimer(),
    "acc": training.accuracy
}

#### 训练模型

In [9]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print("\n\n初始化")
print("-" * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print(f"\n循环 {epoch + 1}/{epochs}")
    print("-" * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



初始化
----------
Valid |     1/1    | loss:    1.8712 | fps:    8.0232 | acc:    0.0000   

循环 1/8
----------
Train |     1/1    | loss:    1.8678 | fps:   24.3647 | acc:    0.1250   
Valid |     1/1    | loss:    1.2026 | fps:   14.7643 | acc:    0.5000   

循环 2/8
----------
Train |     1/1    | loss:    0.2042 | fps:   50.4942 | acc:    1.0000   
Valid |     1/1    | loss:    1.5995 | fps:   12.6769 | acc:    0.5000   

循环 3/8
----------
Train |     1/1    | loss:    0.0218 | fps:   55.0083 | acc:    1.0000   
Valid |     1/1    | loss:    2.5796 | fps:   13.2789 | acc:    0.5000   

循环 4/8
----------
Train |     1/1    | loss:    0.0107 | fps:   47.8045 | acc:    1.0000   
Valid |     1/1    | loss:    3.2210 | fps:   13.6371 | acc:    0.5000   

循环 5/8
----------
Train |     1/1    | loss:    0.0047 | fps:   50.4422 | acc:    1.0000   
Valid |     1/1    | loss:    3.9986 | fps:   13.5192 | acc:    0.5000   

循环 6/8
----------
Train |     1/1    | loss:    0.0015 | fps:   49.3932 |